In [ ]:
# Purpose: Thermal Camera Application in AFP for Maximum Temperature Recognition 
# Date: 05/2025
# UNSW Sydney / EPFL
# Code: Assier de Pompignan Leo

# Note: Change the time.sleep() to suit your computer specification (Default: 0.3)

In [1]:
# Maximum Temperature Recognition model using a thermal range spanning from 24.15°C to 319.33°C
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd
import time
import torch

# Paths
scale_image_path = r"Path_to_\image_scale_1.jpg"
scale_csv_path = r"Path_to_\image_scale_1.csv"
video_path = r"Path_to_\video_dataset\T2.mp4"

# Load YOLO model
model = YOLO(r"Path_to_YOLO_model\runs\segment\train\weights\best.pt")

# Set model to inference mode
model.eval()

# Load the scale image and CSV
scale_image = cv2.imread(scale_image_path)
scale_temps = pd.read_csv(scale_csv_path, header=None).to_numpy()

# Ensure dimensions match
if scale_image.shape[:2] != scale_temps.shape:
    raise ValueError("Mismatch between scale image size and temperature CSV size")

# Create a color-to-temperature mapping
color_temp_map = {}
for y in range(scale_image.shape[0]):
    for x in range(scale_image.shape[1]):
        color = tuple(scale_image[y, x])  # BGR format
        temp = scale_temps[y, x]
        color_temp_map[color] = temp

# Open video
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total number of frames: {total_frames}")

frame_count = 0
global_max_temp = -np.inf
global_max_position = None
global_max_frame = None

# Variables for calculating average temperature
sum_temperature = 0
valid_frames = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    results = model(frame)[0]

    # Init masks
    frame_height, frame_width = frame.shape[:2]
    sample_mask = np.zeros((frame_height, frame_width), dtype=np.uint8)
    roller_mask = np.zeros((frame_height, frame_width), dtype=np.uint8)
    nozzle_mask = np.zeros((frame_height, frame_width), dtype=np.uint8)

    # Get masks and classes
    masks = results.masks.data.cpu().numpy() if results.masks is not None else []
    classes = results.boxes.cls.cpu().numpy().astype(int) if results.boxes is not None else []

    for mask_array, cls in zip(masks, classes):
        binary_mask = (mask_array > 0.5).astype(np.uint8)
        binary_mask = cv2.resize(binary_mask, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)

        if cls == 0:  # sample
            sample_mask = np.maximum(sample_mask, binary_mask)
        elif cls == 1:  # roller
            roller_mask = np.maximum(roller_mask, binary_mask)
        elif cls == 2:  # nozzle
            nozzle_mask = np.maximum(nozzle_mask, binary_mask)

    # Exclude roller and nozzle from sample
    excluded_mask = np.logical_or(roller_mask > 0, nozzle_mask > 0)
    valid_sample_mask = np.logical_and(sample_mask > 0, ~excluded_mask)

    # Find max temperature in current frame
    frame_max_temp = -np.inf
    frame_max_position = None

    for y in range(frame_height):
        for x in range(frame_width):
            if valid_sample_mask[y, x]:
                pixel_color = tuple(frame[y, x])
                if pixel_color in color_temp_map:
                    temp = color_temp_map[pixel_color]
                    if temp > frame_max_temp:
                        frame_max_temp = temp
                        frame_max_position = (x, y)

    print(f"Frame {frame_count}: Max temperature in valid sample region = {frame_max_temp:.2f}°C")

    if frame_max_temp > global_max_temp:
        global_max_temp = frame_max_temp
        global_max_position = frame_max_position
        global_max_frame = frame_count
        print(f"🔥 New global max temperature = {global_max_temp:.2f}°C at {global_max_position}, Frame {global_max_frame}")

    # Add the frame's max temperature to the sum for calculating the average
    if frame_max_temp > -np.inf:  # If a valid temperature was found
        sum_temperature += frame_max_temp
        valid_frames += 1


    # Pause for 0.3 seconds before processing the next frame
    time.sleep(0.3)
    
cap.release()

if global_max_position:
    print("\n--- Final Result ---")
    print(f"Max Temperature: {global_max_temp:.2f}°C")
    print(f"Location: {global_max_position}")
    print(f"Frame Number: {global_max_frame}")
else:
    print("No valid temperature data found in the sample region excluding roller and nozzle.")

# Calculate and print the average temperature
if valid_frames > 0:
    average_temperature = sum_temperature / valid_frames
    print(f"\nAverage Temperature over {valid_frames} valid frames: {average_temperature:.2f}°C")
else:
    print("\nNo valid frames found to calculate the average temperature.")


Total number of frames: 6180

0: 384x480 (no detections), 209.0ms
Speed: 9.3ms preprocess, 209.0ms inference, 113.1ms postprocess per image at shape (1, 3, 384, 480)
Frame 1: Max temperature in valid sample region = -inf°C

0: 384x480 (no detections), 34.7ms
Speed: 3.7ms preprocess, 34.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 480)
Frame 2: Max temperature in valid sample region = -inf°C

0: 384x480 (no detections), 32.9ms
Speed: 3.5ms preprocess, 32.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 480)
Frame 3: Max temperature in valid sample region = -inf°C

0: 384x480 (no detections), 38.2ms
Speed: 4.0ms preprocess, 38.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 480)
Frame 4: Max temperature in valid sample region = -inf°C

0: 384x480 (no detections), 35.7ms
Speed: 3.4ms preprocess, 35.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 480)
Frame 5: Max temperature in valid sample region = -inf°C

0: 384x480 (no d

In [1]:
# Maximum Temperature Recognition model using a thermal range spanning from 23.27°C to 382.82°C
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd
import time
import torch

# Paths
scale_image_path = r"Path_to_\image_scale_2.jpg"
scale_csv_path = r"Path_to_\image_scale_2.csv"
video_path = r"Path_to_\video_dataset\T2.mp4"

# Load YOLO model
model = YOLO(r"Path_to_YOLO_model\runs\segment\train\weights\best.pt")

# Set model to inference mode
model.eval()

# Load the scale image and CSV
scale_image = cv2.imread(scale_image_path)
scale_temps = pd.read_csv(scale_csv_path, header=None).to_numpy()

# Ensure dimensions match
if scale_image.shape[:2] != scale_temps.shape:
    raise ValueError("Mismatch between scale image size and temperature CSV size")

# Create a color-to-temperature mapping
color_temp_map = {}
for y in range(scale_image.shape[0]):
    for x in range(scale_image.shape[1]):
        color = tuple(scale_image[y, x])  # BGR format
        temp = scale_temps[y, x]
        color_temp_map[color] = temp

# Open video
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total number of frames: {total_frames}")

frame_count = 0
global_max_temp = -np.inf
global_max_position = None
global_max_frame = None

# Variables for calculating average temperature
sum_temperature = 0
valid_frames = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    results = model(frame)[0]

    # Init masks
    frame_height, frame_width = frame.shape[:2]
    sample_mask = np.zeros((frame_height, frame_width), dtype=np.uint8)
    roller_mask = np.zeros((frame_height, frame_width), dtype=np.uint8)
    nozzle_mask = np.zeros((frame_height, frame_width), dtype=np.uint8)

    # Get masks and classes
    masks = results.masks.data.cpu().numpy() if results.masks is not None else []
    classes = results.boxes.cls.cpu().numpy().astype(int) if results.boxes is not None else []

    for mask_array, cls in zip(masks, classes):
        binary_mask = (mask_array > 0.5).astype(np.uint8)
        binary_mask = cv2.resize(binary_mask, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)

        if cls == 0:  # sample
            sample_mask = np.maximum(sample_mask, binary_mask)
        elif cls == 1:  # roller
            roller_mask = np.maximum(roller_mask, binary_mask)
        elif cls == 2:  # nozzle
            nozzle_mask = np.maximum(nozzle_mask, binary_mask)

    # Exclude roller and nozzle from sample
    excluded_mask = np.logical_or(roller_mask > 0, nozzle_mask > 0)
    valid_sample_mask = np.logical_and(sample_mask > 0, ~excluded_mask)

    # Find max temperature in current frame
    frame_max_temp = -np.inf
    frame_max_position = None

    for y in range(frame_height):
        for x in range(frame_width):
            if valid_sample_mask[y, x]:
                pixel_color = tuple(frame[y, x])
                if pixel_color in color_temp_map:
                    temp = color_temp_map[pixel_color]
                    if temp > frame_max_temp:
                        frame_max_temp = temp
                        frame_max_position = (x, y)

    print(f"Frame {frame_count}: Max temperature in valid sample region = {frame_max_temp:.2f}°C")

    if frame_max_temp > global_max_temp:
        global_max_temp = frame_max_temp
        global_max_position = frame_max_position
        global_max_frame = frame_count
        print(f"🔥 New global max temperature = {global_max_temp:.2f}°C at {global_max_position}, Frame {global_max_frame}")

    # Add the frame's max temperature to the sum for calculating the average
    if frame_max_temp > -np.inf:  # If a valid temperature was found
        sum_temperature += frame_max_temp
        valid_frames += 1


    # Pause for 0.3 seconds before processing the next frame
    time.sleep(0.3)
    
cap.release()

if global_max_position:
    print("\n--- Final Result ---")
    print(f"Max Temperature: {global_max_temp:.2f}°C")
    print(f"Location: {global_max_position}")
    print(f"Frame Number: {global_max_frame}")
else:
    print("No valid temperature data found in the sample region excluding roller and nozzle.")

# Calculate and print the average temperature
if valid_frames > 0:
    average_temperature = sum_temperature / valid_frames
    print(f"\nAverage Temperature over {valid_frames} valid frames: {average_temperature:.2f}°C")
else:
    print("\nNo valid frames found to calculate the average temperature.")


Total number of frames: 6180

0: 384x480 (no detections), 38.2ms
Speed: 2.6ms preprocess, 38.2ms inference, 9.2ms postprocess per image at shape (1, 3, 384, 480)
Frame 1: Max temperature in valid sample region = -inf°C

0: 384x480 (no detections), 5.1ms
Speed: 1.2ms preprocess, 5.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 480)
Frame 2: Max temperature in valid sample region = -inf°C

0: 384x480 (no detections), 6.0ms
Speed: 0.8ms preprocess, 6.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 480)
Frame 3: Max temperature in valid sample region = -inf°C

0: 384x480 (no detections), 4.9ms
Speed: 0.8ms preprocess, 4.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 480)
Frame 4: Max temperature in valid sample region = -inf°C

0: 384x480 (no detections), 7.9ms
Speed: 1.0ms preprocess, 7.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 480)
Frame 5: Max temperature in valid sample region = -inf°C

0: 384x480 (no detections), 